In [1]:
"""
An example of getting city blocks from city name on OSM.
Local city crs and administrative level must be pecifies.
As an example, for Peterhof city (a part of Saint Petersburg area), the city name and administrative level could be
found here: https://www.openstreetmap.org/relation/367377. Local city crs (UTM zone) could be discovered here: 
https://epsg.io/32636 
"""

from get_blocks import get_blocks

blocks = get_blocks.Blocks_model(city_name="Петергоф", city_crs=32636, city_admin_level=8).get_blocks()

/usr/local/lib/python3.8/dist-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(
2023-03-20 14:25:26.431 | INFO     | get_blocks.get_blocks:_get_city_geometry:91 - City geometry are not provided. Getting water geometry from OSM via overpass turbo
2023-03-20 14:25:27.241 | INFO     | get_blocks.get_blocks:_get_city_geometry:105 - Got city geometry
2023-03-20 14:25:27.246 | INFO     | get_blocks.get_blocks:_get_railways_geometry:198 - Railways geometries are not provided. Getting railways geometries from OSM via overpass turbo
2023-03-20 14:25:27.717 | INFO     | get_blocks.get_blocks:_get_railways_geometry:212 - Got railways geometries
2023-03-20 14:25:27.863 | INFO     | get_blocks.get_blocks:_split_city_geometry:278 - Cut and deleted railways geometries
2023-03-20 14:25:27.866 | INFO     | get_blocks.get

In [2]:
blocks.explore()